In [3]:
# https://habr.com/ru/post/347564/.
import os
os.environ["CUDA_VISIBLE_DEVICES"]="3"
os.environ['THEANO_FLAGS'] = "mode=FAST_RUN,device=cuda,floatX=float32"
import theano
print(theano.config.device, theano.config.floatX)

import keras
import sys
#from google.colab import files # для импорта данных в google colab
#from google.colab import drive # для импорта данных в google colab из google drive
import zipfile # для работы с архивами 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sn
import pickle
import csv
from sklearn.metrics import confusion_matrix, classification_report
import tensorflow as tf
from PIL import Image
import os
import tensorflow as tf

from keras.layers import Input
from keras.applications import xception
from keras.preprocessing import image

from tensorflow.keras.preprocessing import image as im
import time

from keras.preprocessing.image import ImageDataGenerator
from keras.applications import densenet
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, Callback
from keras import regularizers
from keras import backend as K
from sklearn.preprocessing import OneHotEncoder
from keras.utils import to_categorical
from tensorflow.keras.models import load_model  



cuda float32


In [4]:
class_names = ['1', '2', '3', '4', '5']
print(len(class_names))

5


In [5]:
K.set_learning_phase(1)

In [6]:
img_width, img_height = 200, 200
nb_train_samples = 26640
nb_validation_samples = 12630
epochs = 12
batch_size = 8
n_classes = 5

In [7]:
train_data_dir = "/datasets/DTLD/DTLD_crop/new_train/lights"    # дальше там папки, а не сами фотографии!!!!
validation_data_dir = "/datasets/DTLD/DTLD_crop/new_test/lights"

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    #shear_range=0.2,
    zoom_range=0.2,
    #fill_mode = 'constant',
    #cval = 1,
    rotation_range = 5,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip = False,
    vertical_flip = False)

test_datagen = ImageDataGenerator(rescale = 1.0 / 255)

In [8]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size,
    class_mode = 'categorical')


Found 33996 images belonging to 5 classes.
Found 8165 images belonging to 5 classes.


In [9]:
def build_model_Xception():
  base_model = xception.Xception(include_top=True, 
                                      weights = None,
                                      input_tensor = None,
                                      input_shape=(img_width, img_height, 3),
                                      pooling=None,
                                      classes = 5)
  
  for layer in base_model.layers:
    layer.trainable = True
      
  model = Model(inputs = base_model.input, outputs = base_model.output)
  return model

In [10]:
model = build_model_Xception()
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc', 'mse'])

In [9]:
early_stop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, min_delta=1e-4) # остановка обучения, если loss на валидационном множесте улучшается менее чем на 10^-4
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4, verbose=1, min_delta=1e-4)
filepath="weights-lights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"
check = ModelCheckpoint(filepath, monitor = "val_acc", save_best_only = False) # сохранение лучшей (с наибольшим acc на валидационном множестве) сети
callbacks_list = [early_stop, reduce_lr , check]

In [10]:
model_history = model.fit_generator(
    train_generator,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = nb_validation_samples // batch_size,
    callbacks = callbacks_list,
    steps_per_epoch = nb_train_samples // batch_size )

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/12
3330/3330 [==============================] - 6628s 2s/step - loss: 0.5034 - acc: 0.8419 - mean_squared_error: 0.0452 - val_loss: 0.3120 - val_acc: 0.9131 - val_mean_squared_error: 0.0262
Epoch 2/12
3330/3330 [==============================] - 8115s 2s/step - loss: 0.3891 - acc: 0.8690 - mean_squared_error: 0.0365 - val_loss: 0.2269 - val_acc: 0.9278 - val_mean_squared_error: 0.0214
Epoch 3/12
3330/3330 [==============================] - 6352s 2s/step - loss: 0.2954 - acc: 0.8951 - mean_squared_error: 0.0293 - val_loss: 0.1630 - val_acc: 0.9427 - val_mean_squared_error: 0.0167
Epoch 4/12
3330/3330 [==============================] - 5620s 2s/step - loss: 0.2183 - acc: 0.9248 - mean_squared_error: 0.0220 - val_loss: 0.1252 - val_acc: 0.9575 - val_mean_squared_error: 0.0122
Epoch 5/12
3330/3330 [==============================] - 5707s 2s/step - loss: 0.1836 - acc: 0.9354 - mean_squared_

In [26]:
model = load_model('weights-lights-improvement-12-0.98.hdf5')

In [27]:
def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [28]:
def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [29]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['acc', 'mse', precision, recall])

In [30]:
evaluation = model.evaluate_generator(validation_generator, verbose=1)

1021/1021 [==============================] - 1231s 1s/step - loss: 0.1043 - acc: 0.9671 - mean_squared_error: 0.0101 - precision: 0.9722 - recall: 0.9635


In [31]:
model.metrics_names

['loss', 'acc', 'mean_squared_error', 'precision', 'recall']

In [32]:
evaluation

[0.10426906616989612, 0.9670545, 0.010063752, 0.97220844, 0.9635162]